## Convert the COPEPOD data from csv to nc files. 

In [ ]:
import xarray as xr
import pandas as pd
from glob import glob
import numpy as np

In [ ]:
files = glob('data/*.csv')
files = sorted(files)

In [ ]:
doy = {'01': 15., '02': 45., '03': 74., '04': 104., '05': 135., '06': 166.,
    '07': 196., '08': 227., '09': 258., '10': 288., '11': 319., '12': 349.,
    '13': 90., '14': 181., '15': 273., '16': 365., '00': 0.}

In [ ]:
lon = np.arange(-180+0.125,180.,0.25)
lat = np.arange(-90+0.125,90.,0.25)
coords = {'lat': np.array(list(lat) * len(lon)), 'lon': np.repeat(lon, len(lat))}

In [ ]:
df = pd.DataFrame.from_dict(coords)

In [ ]:
for f in files:
    print 'Processing file %s'%f
    
    d = pd.read_csv(f).iloc[:,0:4]
    d.columns = ['lon', 'lat', 'n', 'carbon_biomass']

    d = pd.merge(df, d.reset_index(), how='outer')

    d['time'] = doy[f[26:28]]
    d = d.set_index(['time', 'lat', 'lon'])
    d = d.loc[:,['n', 'carbon_biomass']]

    xd = xr.Dataset.from_dataframe(d)
    xd['n'].attrs={'units': 'nan', 'long_name' : 'Number of observations'}
    xd['carbon_biomass'].attrs={'units': 'mg-C m^-3', 'long_name': 'Total Carbon Biomass'}
    
    xd.to_netcdf(f.replace('.csv','.nc'))

## Regrid to POP

In [ ]:
#! /usr/bin/env python
#BSUB -P NCGD0011
#BSUB -W 24:00
#BSUB -n 1
#BSUB -J gen_POP_grid_files
#BSUB -o logs/regrid.%J
#BSUB -e logs/regrid.%J
#BSUB -q geyser
#BSUB -N

import os
from subprocess import call
from glob import glob
from regrid import regrid
import xarray as xr

dst_grid = 'POP_gx1v7'
interp_method = 'conserve'
postfill_opt = 'none'
prefill_opt = 'none'

src_grid = 'latlon_0.25x0.25_180W'
variables = ['carbon_biomass','n']

files = sorted(glob('data/*.nc'))

for file_in in files:

    print file_in

    for dst_grid in ['POP_gx1v7','POP_gx3v7']:
        vert_grid_file = regrid.vert_grid_file(dst_grid)

        wgtFile = regrid.wgt_file(src_grid,dst_grid,interp_method)
        if not os.path.exists(wgtFile):
            print('missing weight file: %s'%wgtFile)
            exit(1)

        file_out = file_in.replace('.nc','_'+dst_grid+'.nc')

        print('\t-->%s'%file_out)
        outfile_opt = 'create'
        for varname_in in variables:
            varname_out = varname_in
            print('\t\t%s'%varname_in)
            ok = regrid.regrid_var(wgtFile = wgtFile,
                                   fname_in = file_in,
                                   varname_in = varname_in,
                                   time_coordname = 'time',
                                   depth_coordname = 'none',
                                   vert_grid_file = vert_grid_file,
                                   fname_out = file_out,
                                   varname_out = varname_out,
                                   src_grid = src_grid,
                                   dst_grid = dst_grid,
                                   postfill_opt = postfill_opt,
                                   prefill_opt = prefill_opt,
                                   outfile_opt = outfile_opt)
            outfile_opt = 'append'
        print
